In [2]:
import py2neo
import tqdm
import numpy as np
import time
import pandas as pd

1. You will continue to work on the MCU network for a while. 
    Who are the 10 fastest in the MCU ? Are they related to each other ?

In [7]:
query = """MATCH (a:Superhero)
WHERE exists(a.Speed)
RETURN a
ORDER BY a.Speed DESC
LIMIT 10"""
data = graph.run(query)
for _ in data:
    print(_)


Node('Superhero', Combat=56, Durability=84, Intelligence=75, Power=100, Speed=100, Strength=67, Total=482, id_=1009184, name='Black Bolt')
Node('Superhero', Combat=85, Durability=110, Intelligence=88, Power=100, Speed=100, Strength=85, Total=568, id_=1010764, name='Stardust')
Node('Superhero', Combat=56, Durability=60, Intelligence=63, Power=57, Speed=100, Strength=28, Total=364, id_=1009524, name='Quicksilver')
Node('Superhero', Combat=25, Durability=100, Intelligence=38, Power=100, Speed=100, Strength=60, Total=423, id_=1009477, name='Nova')
Node('Superhero', Combat=85, Durability=100, Intelligence=69, Power=100, Speed=92, Strength=100, Total=546, id_=1009664, name='Thor')
Node('Superhero', Combat=32, Durability=101, Intelligence=63, Power=100, Speed=84, Strength=100, Total=480, id_=1009592, name='Silver Surfer')
Node('Superhero', Combat=70, Durability=100, Intelligence=100, Power=100, Speed=83, Strength=100, Total=553, id_=1009312, name='Galactus')
Node('Superhero', Combat=80, Durab

In [25]:
superheroes = ['Black Bolt', 'Stardust', 'Quicksilver', 'Nova', 'Thor', 'Silver Surfer', 'Galactus', 'Dormammu', 'Ego', 'Northstar']
heros_n = ', '.join([f'"{hero}"' for hero in superheroes])

In [28]:
# Requête Cypher pour compter le nombre de relations pour chaque super-héros spécifique
query = f"""MATCH (a:Superhero)-[r:RELATION]-(b:Superhero) 
WHERE a.name IN [{heros_n}] AND b.name IN [{heros_n}] 
RETURN a.name AS superhero, count(r) AS num_relations"""

# Exécutez la requête et convertissez les résultats en DataFrame
data = graph.run(query).to_data_frame()

# Afficher le DataFrame
print(data)


       superhero  num_relations
0       Galactus              9
1  Silver Surfer              8
2           Thor              8
3    Quicksilver              8
4       Dormammu              4
5            Ego              5
6           Nova              6
7     Black Bolt              5
8      Northstar              3
9       Stardust              2


In [8]:
query = """MATCH (n:Superhero)-[r]-(m:Superhero)
WHERE n.name IN ['Black Bolt', 'Stardust', 'Quicksilver', 'Nova', 'Thor', 'Silver Surfer', 'Galactus', 'Dormammu', 'Ego', 'Northstar']
AND m.name IN ['Black Bolt', 'Stardust', 'Quicksilver', 'Nova', 'Thor', 'Silver Surfer', 'Galactus', 'Dormammu', 'Ego', 'Northstar']
RETURN n.name, m.name"""

results = graph.run(query)

for record in results:
    print(record)

'Ego'	'Galactus'
'Dormammu'	'Galactus'
'Black Bolt'	'Galactus'
'Northstar'	'Galactus'
'Nova'	'Galactus'
'Stardust'	'Galactus'
'Silver Surfer'	'Galactus'
'Quicksilver'	'Galactus'
'Thor'	'Galactus'
'Ego'	'Silver Surfer'
'Nova'	'Silver Surfer'
'Quicksilver'	'Silver Surfer'
'Black Bolt'	'Silver Surfer'
'Galactus'	'Silver Surfer'
'Dormammu'	'Silver Surfer'
'Stardust'	'Silver Surfer'
'Thor'	'Silver Surfer'
'Northstar'	'Thor'
'Ego'	'Thor'
'Silver Surfer'	'Thor'
'Nova'	'Thor'
'Black Bolt'	'Thor'
'Galactus'	'Thor'
'Quicksilver'	'Thor'
'Dormammu'	'Thor'
'Dormammu'	'Quicksilver'
'Ego'	'Quicksilver'
'Nova'	'Quicksilver'
'Galactus'	'Quicksilver'
'Black Bolt'	'Quicksilver'
'Northstar'	'Quicksilver'
'Silver Surfer'	'Quicksilver'
'Thor'	'Quicksilver'
'Quicksilver'	'Dormammu'
'Galactus'	'Dormammu'
'Thor'	'Dormammu'
'Silver Surfer'	'Dormammu'
'Galactus'	'Ego'
'Thor'	'Ego'
'Nova'	'Ego'
'Silver Surfer'	'Ego'
'Quicksilver'	'Ego'
'Black Bolt'	'Nova'
'Ego'	'Nova'
'Galactus'	'Nova'
'Silver Surfer'	'Nova'
'Qui

2. How many have collaborated atleast more than 10 times with another superhero?

In [5]:
import py2neo
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="mcu")

query = """MATCH (a:Superhero)-[r:RELATION]-(b:Superhero)
WHERE r.n_collab > 10
WITH a
MATCH (a)-[r:RELATION]-(b:Superhero)
WHERE r.n_collab >= 10
RETURN COUNT(DISTINCT a)"""

result = graph.run(query).evaluate()
print(f"Le nombre de héros qui ont collaboré plus de 10 fois avec d'autres héros est de {result} ")

Le nombre de héros qui ont collaboré plus de 10 fois avec d'autres héros est de 528 


3. Did the weakest of the MCU collaborated with the strongest ?

In [10]:
import py2neo
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="mcu")

# Le plus faible
query_weakest = """MATCH (n:Superhero)
WHERE (n.Total) IS NOT NULL
RETURN n.name
ORDER BY n.Total
LIMIT 1"""
result_weakest = graph.run(query_weakest).evaluate()

# Le plus fort
query_strongest = """MATCH (n:Superhero)
WHERE (n.Total) IS NOT NULL
RETURN n.name
ORDER BY n.Total DESC
LIMIT 1"""
result_strongest = graph.run(query_strongest).evaluate()

print(f"Le héros le plus fort est {result_strongest},\ntandis que le plus faible est {result_weakest}")

Le héros le plus fort est Stardust,
tandis que le plus faible est Lady Deathstrike


In [11]:
# Chercher une éventuelle collaboration
query_relation = f"""MATCH (a:Superhero {{name: "{result_weakest}"}})-[r:RELATION]-(b:Superhero {{name: "{result_strongest}"}})
RETURN COUNT(r)"""
result_relation = graph.run(query_relation).evaluate()

if result_relation > 0:
    print(f"{result_weakest} a collaboré avec {result_strongest}")
else:
    print(f"{result_weakest} n'a pas collaboré avec {result_strongest}")

Lady Deathstrike n'a pas collaboré avec Stardust


4. Give a stats that interests you !
If Thanos was able to snap fingers, is it because those who can beat him
don't appear in enough comics ?

In [41]:
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="mcu")

query = """MATCH (n:Superhero)
WHERE (n.Total) IS NOT NULL
RETURN n.name, n.Total
ORDER BY n.Total DESC
LIMIT 10"""

results = graph.run(query)

for _ in results:
    print(_)

'Stardust'	568
'Galactus'	553
'Dormammu'	546
'Thor'	546
'Jean Grey'	517
'Jean Grey'	517
'Onslaught'	505
'Apocalypse'	493
'Iron Man'	492
'Thanos'	485


In [42]:
import pandas as pd

# Liste des super-héros spécifiques
superheroes = ["Stardust", "Galactus", "Dormammu", "Thor", "Jean Grey", "Onslaught", "Apocalypse", "Iron Man", "Thanos"]

# Construire une chaîne de caractères pour filtrer les super-héros spécifiques dans la requête Cypher
heroes_filter = ', '.join([f'"{hero}"' for hero in superheroes])

# Requête Cypher pour compter le nombre de relations pour chaque super-héros spécifique
query = f"""MATCH (a:Superhero)-[r:RELATION]-(b:Superhero) 
WHERE a.name IN [{heroes_filter}] AND b.name IN [{heroes_filter}] 
RETURN a.name AS superhero, count(r) AS num_relations"""

# Exécutez la requête et convertissez les résultats en DataFrame
data = graph.run(query).to_data_frame()

# Afficher le DataFrame
print(data)


    superhero  num_relations
0    Iron Man              8
1    Galactus              7
2   Jean Grey             11
3        Thor              8
4      Thanos              6
5    Dormammu              4
6   Onslaught              6
7  Apocalypse              4
8    Stardust              2


In [43]:
query=f"""MATCH (s:Superhero)-[:RELATION]-(other:Superhero)
WITH s, count(*) AS num_relations
ORDER BY num_relations DESC
LIMIT 10
RETURN s.name AS superhero, num_relations
"""
# Exécutez la requête et convertissez les résultats en DataFrame
data = graph.run(query).to_data_frame()

# Afficher le DataFrame
print(data)


         superhero  num_relations
0        Wolverine            648
1       Spider-Man            639
2            X-Men            607
3         Iron Man            602
4  Captain America            566
5         Avengers            532
6             Thor            508
7            Beast            504
8          Cyclops            488
9             Hulk            471


La plupart des héros qui sont puissants n'étaient tout simplement pas là pour défaire Thanos.

5. Create a new db called "master". Create a graph from the M1 class with name and random age between 20 and 30, Group of 3 that collab (undirected relation).

In [8]:
import random

graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="master")
graph.delete_all()
noms = [
    "Adrien",
    "Alexis",
    "Alexandre",
    "Antoine",
    "Arthur",
    "Baptiste",
    "Benjamin",
    "Clément",
    "Corentin",
    "Damien",
    "David",
    "Dylan",
    "Eliot",
    "Enzo",
    "Maëlys",
    "Manon",
    "Marine",
    "Mathilde",
    "Mélanie",
    "Naomi",
    "Natacha",
    "Nina",
    "Olivia",
    "Pauline"
]
personnes = [py2neo.Node("M1", name=nom, age=random.randint(20, 30)) for nom in noms]
transaction = graph.begin()

for personne in personnes:
    transaction.create(personne)

transaction.commit()
print(graph.exists(personne))

C:\Users\carlf\AppData\Local\Temp\ipykernel_17184\1188914590.py:37: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  transaction.commit()


True


In [10]:
# Créer des groupes de 3 en supprimant à chaque fois 
#les élèves dans les groupes formés (afin de ne pas avoir l'élève A dans 2 groupes)
groupes = []
while len(personnes) >= 3:
    groupe = random.sample(personnes, 3)
    groupes.append(groupe)
    for personne in groupe:
        personnes.remove(personne)

for groupe in groupes:
    for i, j in zip(groupe, groupe[1:] + [groupe[0]]):
        relation = py2neo.Relationship(i, "collab", j)
        graph.create(relation)
print("Relations créées :")
for record in graph.run("MATCH ()-[r:collab]-() RETURN r"):
    print(record)

Relations créées :
collab(Node('M1', age=21, name='Manon'), Node('M1', age=26, name='Adrien'))
collab(Node('M1', age=26, name='Adrien'), Node('M1', age=23, name='Baptiste'))
collab(Node('M1', age=21, name='Alexis'), Node('M1', age=29, name='Clément'))
collab(Node('M1', age=25, name='David'), Node('M1', age=21, name='Alexis'))
collab(Node('M1', age=22, name='Alexandre'), Node('M1', age=22, name='Maëlys'))
collab(Node('M1', age=29, name='Antoine'), Node('M1', age=22, name='Alexandre'))
collab(Node('M1', age=29, name='Antoine'), Node('M1', age=22, name='Alexandre'))
collab(Node('M1', age=22, name='Maëlys'), Node('M1', age=29, name='Antoine'))
collab(Node('M1', age=27, name='Nina'), Node('M1', age=25, name='Arthur'))
collab(Node('M1', age=25, name='Arthur'), Node('M1', age=26, name='Eliot'))
collab(Node('M1', age=23, name='Baptiste'), Node('M1', age=21, name='Manon'))
collab(Node('M1', age=26, name='Adrien'), Node('M1', age=23, name='Baptiste'))
collab(Node('M1', age=24, name='Benjamin'), 

6. Get the age for people YOU collaborate with.


In [12]:
from py2neo import Graph


graph = Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="master")


query = """
MATCH (dylan:M1 {name: 'Dylan'})-[:collab]-(personne:M1)
RETURN personne.name AS name, personne.age AS age
"""


print("J'ai collaboré avec les personnes suivantes, et voici leur âge :")
for record in graph.run(query):
    print(record["name"], ":", record["age"])


J'ai collaboré avec les personnes suivantes, et voici leur âge :
Corentin : 25
Pauline : 29


7. Get nodes with age > 25.

In [21]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="master")


query = """
MATCH (personne:M1)
WHERE personne.age >= 25
RETURN personne.name AS name, personne.age AS age
"""

# Exécution de la requête et impression des résultats
print("Personne(s) avec un âge supérieur à 25 :")
i = 0 
for record in graph.run(query):
    print(record["name"], ":", record["age"])
    i += 1
print(f"\nIls sont {i} au total !")


Personne(s) avec un âge supérieur à 25 :
Adrien : 26
Antoine : 29
Arthur : 25
Clément : 29
Corentin : 25
David : 25
Eliot : 26
Enzo : 27
Mélanie : 25
Naomi : 27
Nina : 27
Olivia : 29
Pauline : 29

Ils sont 13 au total !


Real world problem:


9: Get the density of degrees of the collaboration network

10: Do a for loop for i in range(500) for each i remove nodes with a degree less than i.

8. Using the downloaded data in TODO 11 of chapter II, create a collaboration network between authors for papers after 2021 (same author if same name). Insert it in neo4j in a db called "arxiv".

In [ ]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="arxiv")

In [29]:
import json
from py2neo import Graph, Node, Relationship

# Chemin vers le fichier JSON
json_file_path = "C:\\Users\\carlf\\OneDrive\\Bureau\\NoSQL_SQL\\data_world.json"

# Connexion à la base de données Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="arxiv")

# Charger les données JSON depuis le fichier
with open(json_file_path, "r") as file:
    data = json.load(file)

# Démarrer une transaction
tx = graph.begin()

# Parcourir les données pour créer le réseau de collaboration
for entry in data:
    # Vérifier si l'article a une année de publication
    if "year" in entry and entry["year"] > 2021:
        print(f"Processing entry: {entry['title']}")

        # Récupérer les auteurs de l'article s'ils existent
        if "authors" in entry:
            authors = entry["authors"]
            print(f"Authors: {authors}")

            # Créer des nœuds pour chaque auteur s'ils n'existent pas déjà
            author_nodes = {}
            for author in authors:
                if author not in author_nodes:
                    author_nodes[author] = Node("Author", name=author)
                    tx.merge(author_nodes[author], "Author", "name")
                    print(f"Node created for author: {author}")

            # Créer des relations de collaboration entre les auteurs
            for i, author1 in enumerate(authors):
                for j, author2 in enumerate(authors):
                    if i != j:
                        # Créer une relation de collaboration si elle n'existe pas déjà
                        if not graph.exists(Relationship(author_nodes[author1], "COLLAB", author_nodes[author2])):
                            tx.create(Relationship(author_nodes[author1], "COLLAB", author_nodes[author2]))
                            print(f"Collaboration relationship created between {author1} and {author2}")

# Valider la transaction (commit)
tx.commit()

print("Collaboration network created and inserted into Neo4j.")


Collaboration network created and inserted into Neo4j.


C:\Users\carlf\AppData\Local\Temp\ipykernel_17184\27898242.py:46: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Real World application 3
You will work again on pubmed_last_100k.json. The goal in this homework is to insert collaboration data in Neo4j and then study it using cypher queries. The following questions are meant to guide you through the process.

1. Each paper has a list of author and affiliations. The first step is to create a node per author with a list of affiliation as attribute. If a name appears multiple times consider it as a single person. Create an edge between two authors if they are on the same paper. You can use any kind of operation to clean the data before inserting, you decide which preprocessing step is the most convenient.

2. How many authors have more than 2 affiliations ?

3. Create a density of node degrees. Do you observe a power scale law ? What does it mean ?

4. Which country has seen the most international collaboration ?

5. Who are the top 10 collaborators and what is their affiliation ?

6. Sandbox exercise: think of a problematic and try to answer it.